# Upscaling
To achieve upscaling of the low-res image, we need to co-register the Sentinel data with LiDAR data

In [2]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

def coregister_sentinel_to_lidar(sentinel_path, lidar_path, output_path):
    """
    Coregisters (aligns) Sentinel imagery to match the LiDAR raster's
    resolution, extent, and coordinate system.
    
    Args:
        sentinel_path (str): Path to the Sentinel raster (e.g., .tif).
        lidar_path (str): Path to the LiDAR raster (e.g., .tif).
        output_path (str): Output path for the coregistered Sentinel raster.
    """
    # 1. Open LiDAR as the 'reference' dataset
    with rasterio.open(lidar_path) as lidar_src:
        # Read profile (metadata), will use this for alignment
        lidar_profile = lidar_src.profile
        lidar_crs = lidar_src.crs
        lidar_transform = lidar_src.transform
        lidar_bounds = lidar_src.bounds

    # 2. Open Sentinel as the 'source' dataset
    with rasterio.open(sentinel_path) as sentinel_src:
        sentinel_profile = sentinel_src.profile
        sentinel_crs = sentinel_src.crs
        sentinel_transform = sentinel_src.transform
        sentinel_bounds = sentinel_src.bounds

        # 3. Compute the transform/shape needed to match LiDAR's resolution & CRS
        #    By default, we can reproject Sentinel to exactly match LiDAR’s grid.
        transform, width, height = calculate_default_transform(
            src_crs=sentinel_crs,
            dst_crs=lidar_crs,
            width=sentinel_src.width,
            height=sentinel_src.height,
            left=sentinel_bounds.left,
            bottom=sentinel_bounds.bottom,
            right=sentinel_bounds.right,
            top=sentinel_bounds.top,
            # or you could use `*lidar_bounds` if you specifically want to 
            # match the LiDAR's bounding box. Depends on your region of interest.
        )

        # 4. Update the profile to match LiDAR's specs
        #    (same CRS, resolution, transform, etc.)
        aligned_profile = sentinel_profile.copy()
        aligned_profile.update({
            'crs': lidar_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # 5. Create the output dataset (coregistered Sentinel)
        with rasterio.open(output_path, 'w', **aligned_profile) as dst:
            # Reproject each band from Sentinel
            for i in range(1, sentinel_profile['count'] + 1):
                # Re-read each band from the original Sentinel file
                sentinel_band = sentinel_src.read(i)

                reproject(
                    source=sentinel_band,
                    destination=rasterio.band(dst, i),
                    src_transform=sentinel_transform,
                    src_crs=sentinel_crs,
                    dst_transform=transform,
                    dst_crs=lidar_crs,
                    resampling=Resampling.bilinear
                    # or nearest/cubic, depending on preference
                )

    print(f"Coregistered Sentinel written to: {output_path}")

In [4]:
# ------------------- USAGE EXAMPLE -------------------
if __name__ == "__main__":
    # Replace with your actual file paths
    sentinel_tif = "./HackathonData/20230215_SE2B_CGG_GBR_MS4_L3_BGRN.tif"
    lidar_tif = "./HackathonData/DSM_TQ0075_P_12757_20230109_20230315.tif"
    output_tif = "./coregistered_data.tif" # pwd
    
    coregister_sentinel_to_lidar(sentinel_tif, lidar_tif, output_tif)


Coregistered Sentinel written to: ./coregistered_data.tif


In [6]:
import os
from osgeo import gdal

def coregister_sentinel_to_lidar(
    sentinel_path: str,
    lidar_path: str,
    output_path: str,
    resampling_method: str = 'bilinear'
):
    """
    Reproject (warp) the Sentinel raster to match LiDAR's resolution, extent, and CRS.

    :param sentinel_path: Path to the 3-band Sentinel file (e.g. .tif).
    :param lidar_path: Path to the 1-band LiDAR file (e.g. .tif).
    :param output_path: Output path for the resulting coregistered Sentinel file.
    :param resampling_method: GDAL resampling method: e.g. 'near', 'bilinear', 'cubic', 'lanczos', etc.
    """

    # 1. Open the LiDAR file to extract CRS, bounds, resolution
    lidar_ds = gdal.Open(lidar_path, gdal.GA_ReadOnly)
    if not lidar_ds:
        raise RuntimeError(f"Could not open LiDAR file: {lidar_path}")

    # Read LiDAR’s projection (CRS)
    lidar_proj = lidar_ds.GetProjection()
    # Read the geotransform (top-left x, w-e pixel resolution, rotation, top-left y, rotation, n-s pixel resolution)
    lidar_transform = lidar_ds.GetGeoTransform()

    # Extract bounding coordinates (upper-left x, width, rotation, upper-left y, rotation, height)
    ulx = lidar_transform[0]  # top-left x
    xres = lidar_transform[1]  # pixel width
    xrot = lidar_transform[2]  # rotation (typically 0)
    uly = lidar_transform[3]  # top-left y
    yrot = lidar_transform[4]  # rotation (typically 0)
    yres = lidar_transform[5]  # pixel height (negative in north-up images)

    # Calculate the lower-right corner
    lrx = ulx + (lidar_ds.RasterXSize * xres)
    lry = uly + (lidar_ds.RasterYSize * yres)

    # 2. Close LiDAR dataset if you want to free resources
    lidar_ds = None

    # 3. Prepare gdal.Warp options
    # Output bounds -> [minX, minY, maxX, maxY]
    # Because yres is negative, top-left is (ulx, uly), bottom-right is (lrx, lry).
    warp_options = gdal.WarpOptions(
        format='GTiff',
        dstSRS=lidar_proj,              # match LiDAR projection
        xRes=abs(xres),                 # resolution in x (positive value)
        yRes=abs(yres),                 # resolution in y (positive value)
        outputBounds=[ulx, lry, lrx, uly],  # match LiDAR bounding box
        resampleAlg=resampling_method,  # e.g. 'bilinear', 'cubic', 'near', etc.
        multithread=True,               # speed up if you have multiple cores
        srcNodata=None,                 # if you have nodata value
        dstNodata=None
    )

    # 4. Run gdal.Warp to produce the new raster
    result = gdal.Warp(
        destNameOrDestDS=output_path,
        srcDSOrSrcDSName=sentinel_path,
        options=warp_options
    )

    # 5. Close the dataset to ensure it's written to disk
    if result:
        result = None
    print(f"Coregistered Sentinel saved to {output_path}")



In [8]:
# ---------------- USAGE EXAMPLE -------------------
if __name__ == "__main__":
    sentinel_file = "./HackathonData/20230215_SE2B_CGG_GBR_MS4_L3_BGRN.tif"
    lidar_file = "./HackathonData/DSM_TQ0075_P_12757_20230109_20230315.tif"
    output_file = "./coregistered_data.tif" # pwd

    coregister_sentinel_to_lidar(
        sentinel_path=sentinel_file,
        lidar_path=lidar_file,
        output_path=output_file,
        resampling_method='bilinear'
    )


/home/ianyang/micromamba/envs/BioHackers/lib/python3.13/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
Warning 1: PROJ: proj_create_from_database: Open of /home/ianyang/micromamba/envs/BioHackers/share/proj failed
Warning 1: PROJ: proj_create_from_name: Open of /home/ianyang/micromamba/envs/BioHackers/share/proj failed
Warning 1: The definition of projected CRS EPSG:27700 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.


TypeError: Warp() missing 1 required positional argument: 'srcDSOrSrcDSTab'

In [9]:
import os
from osgeo import gdal

def coregister_sentinel_to_lidar(
    sentinel_path: str,
    lidar_path: str,
    output_path: str,
    resampling_method: str = 'bilinear'
):
    """
    Reproject (warp) the Sentinel raster to match LiDAR's resolution, extent, and CRS.

    :param sentinel_path: Path to the 3-band Sentinel file (e.g., .tif).
    :param lidar_path: Path to the 1-band LiDAR file (e.g., .tif).
    :param output_path: Output path for the resulting coregistered Sentinel file.
    :param resampling_method: GDAL resampling method: e.g. 'near', 'bilinear', 'cubic', 'lanczos', etc.
    """

    # 1. Open the LiDAR to read CRS, extent, and resolution
    ds_lidar = gdal.Open(lidar_path, gdal.GA_ReadOnly)
    if ds_lidar is None:
        raise RuntimeError(f"Could not open LiDAR file: {lidar_path}")

    lidar_proj = ds_lidar.GetProjection()      # CRS / projection
    lidar_transform = ds_lidar.GetGeoTransform()  # (ulx, xres, xrot, uly, yrot, yres)
    xsize = ds_lidar.RasterXSize
    ysize = ds_lidar.RasterYSize

    # Extract bounding coords
    ulx = lidar_transform[0]
    uly = lidar_transform[3]
    xres = lidar_transform[1]      # pixel width
    yres = lidar_transform[5]      # pixel height (likely negative)
    lrx = ulx + xsize * xres
    lry = uly + ysize * yres

    ds_lidar = None  # close the dataset

    # 2. Warp: direct style
    #
    # We pass sentinel_path as the second argument (the "source"), 
    # and specify all warp parameters (including projection, resolution, bounding box).
    # This usage avoids the "missing 1 required positional argument" error in some GDAL versions.
    #
    # Note: For multi-band Sentinel, ensure the source file truly has 3 bands. Otherwise, 
    # the output will be single-band if the source is only single-band.
    result = gdal.Warp(
        destNameOrDestDS=output_path,
        srcDSOrSrcDSTab=sentinel_path,  # Could also pass [sentinel_path] if your GDAL version requires a list
        format='GTiff',
        dstSRS=lidar_proj,                            # Match LiDAR's CRS
        xRes=abs(xres),                               # 1-m resolution if xres=1.0 (and so on)
        yRes=abs(yres),                               # The absolute value of yres
        outputBounds=[ulx, lry, lrx, uly],            # Align bounding box with LiDAR
        resampleAlg=resampling_method,                # bilinear, cubic, near, etc.
        dstNodata=None
    )

    if not result:
        raise RuntimeError("Warp failed or returned None. Check your GDAL installation and inputs.")
    
    # Close to flush to disk
    result = None
    print(f"Coregistered Sentinel saved to: {output_path}")



In [10]:

# -------------- USAGE EXAMPLE --------------
if __name__ == "__main__":
    sentinel_file = "./HackathonData/20230215_SE2B_CGG_GBR_MS4_L3_BGRN.tif"
    lidar_file = "./HackathonData/DSM_TQ0075_P_12757_20230109_20230315.tif"
    output_file = "./coregistered_data.tif"

    coregister_sentinel_to_lidar(
        sentinel_path=sentinel_file,
        lidar_path=lidar_file,
        output_path=output_file,
        resampling_method='bilinear'
    )


Warning 1: PROJ: proj_create_from_database: Open of /home/ianyang/micromamba/envs/BioHackers/share/proj failed
Warning 1: PROJ: proj_create_from_name: Open of /home/ianyang/micromamba/envs/BioHackers/share/proj failed
Warning 1: The definition of projected CRS EPSG:27700 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
Warning 1: 20230215_SE2B_CGG_GBR_MS4_L3_BGRN.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: PROJ: proj_create_from_database: Open of /home/ianyang/micromamba/envs/BioHackers/share/proj failed
Warning 1: The definition of projected CRS EPSG:27700 got from GeoTIFF keys is not the

Coregistered Sentinel saved to: ./coregistered_data.tif
